In [1]:
import pickle
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import scipy.signal as signal
from scipy.signal import hilbert
import os

In [2]:
%matplotlib qt
# to make plots interactive

In [3]:
file_path = r"C:\EEG DATA\FL_label_data.pickle"
# added r in front of file path to make it a raw string, to make sure that \ is not interpreted as a newline character

# open the pickle file
with open(file_path, "rb") as file:
    label_data = pickle.load(file)

# show the label_data type
print(type(label_data))

<class 'dict'>


In [4]:
def extract_onsets(label_data):
    onset_dict = {}
    for key, value in label_data.items():
        labels = np.atleast_1d(value['label'])
        onsets = np.atleast_1d(value['onset'])
        # to ensure that labels and onsets are treated as array
        # because subsequently using np.where
        indices = np.where((labels == 1) | (labels == 2))[0]
        # returns indices where the label is 1 (N2) or 2 (N3)
        if indices.size > 0 and np.all(indices < len(onsets)):
            # to ensure that no out-of-bounds error
            selected_onsets = onsets[indices]
            # retrieve onset value corresponding to label 1 or 2
            onset_dict[key] = selected_onsets
            # save extracted onset under correct key in dict
            #print(f"Key: {key}, Onset values for labels 1 (N2) and 2 (N3): {', '.join(map(str, selected_onsets))}")
        else:
            print(f"Key: {key}, Warning: The indices do not match")
    return onset_dict
    # returning the onset_dict and what you're printing
    # should I be only returning what is supposed to be printed? or maybe only the dict, since already has commas?

label_data_onsets = extract_onsets(label_data)

Key: 083, Warning: The indices do not match
Key: 084, Warning: The indices do not match
Key: 086, Warning: The indices do not match
Key: 038, Warning: The indices do not match
Key: 040, Warning: The indices do not match
Key: 043, Warning: The indices do not match
Key: 045, Warning: The indices do not match
Key: 062, Warning: The indices do not match
Key: 065, Warning: The indices do not match
Key: 069, Warning: The indices do not match
Key: 073, Warning: The indices do not match
Key: 085, Warning: The indices do not match
Key: 091, Warning: The indices do not match


In [5]:
def group_by_increment(onset_values, increment=30):
    
    groups = []
    # will be a list of lists
    current_group = [float(onset_values[0])]
    # initializes this list with the first value from onset_values (the input)
    
    for i in range(1, len(onset_values)):
        # loops through all the onset values
        if onset_values[i] - onset_values[i - 1] == increment:
            # if i = 1, if onset_values[1] - onset_values[0] == 30
            current_group.append(float(onset_values[i]))
            # add the value at current index
        else:
            # if not a difference of 30
            # means you've reached the end of that sublist
            if len(current_group) >= 1:
                # if there is at least a value in that group
                groups.append(current_group)
                # add the sublist to the big list
            current_group = [float(onset_values[i])]
            # starts a new current group with the new value at the current index
    
    if len(current_group) >= 1:
        groups.append(current_group)
    # once you exit the group, if the last current_group contains more than one value
    # then you can add it to group
    # to make sure that last sequence is not left out
    
    return groups

In [6]:
def extract_segments(raw, groups):
    raw_segments = []
    # empty list to store the extracted EEG segments
    #max_time = raw.times[-1]
    
    for group in groups:
        start = group[0]
        # start = first value in group
        #stop = min(group[-1], max_time) 
        stop = group[-1]
        # stop = last value in group

        #if start >= max_time:
            #continue
        # takes the smaller of the two values
        segment = raw.copy().crop(tmin=start, tmax=stop)
        raw_segments.append(segment)
    
    return raw_segments

In [7]:
#segments_033 = extract_segments(participant_033_raw, groups_033)

def concatenate_segments(segments):
    if not segments:
        return None
    combined_raw = mne.concatenate_raws(segments)
    # concatenates raw segments as if they were continuous
    # boundaries of the raw files are annotated bad
    #combined_raw_033.pick(["Fz"]).filter(l_freq=0.1, h_freq=40).plot()
    combined_raw.set_eeg_reference(ref_channels = ['M1', 'M2'])
    combined_raw.apply_function(lambda x: x * 1e6, picks='eeg')
    combined_raw.pick(["Fz"])

    return combined_raw

In [8]:
def load_eeg_data(filepath):
    return mne.io.read_raw_brainvision(filepath, preload=True)

In [9]:
participants = ["020", "033", "046", "067", "081"]
base_dir = r"C:\EEG DATA"

for pid in participants:

    print(f"Processing {pid}...")

    # Load raw EEG
    filepath = os.path.join(base_dir, pid, "eeg", "TMR.vhdr")
    raw = load_eeg_data(filepath)

    # Run analysis pipeline
    if pid in label_data_onsets:
        onset_values = label_data_onsets[pid]
        groups = group_by_increment(onset_values)
        segments = extract_segments(raw, groups)
        combined_raw = concatenate_segments(segments)

        if combined_raw is not None:
            save_dir = os.path.join(base_dir, pid, "concatenated")
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f"{pid}_concatenated_raw.fif")
            combined_raw.save(save_path, overwrite=True)
            print(f"Saved to {save_path}")
        else:
            print(f"No valid segments to save for {pid}")
    else:
        print(f"No onset data for {pid}")

# example of save path (participant 033)
# C:\EEG DATA\033\concatenated\033_concatenated_raw.fif
    

Processing 020...
Extracting parameters from C:\EEG DATA\020\eeg\TMR.vhdr...
Setting channel info structure...
Reading 0 ... 11188539  =      0.000 ... 22377.078 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Overwriting existing file.
Writing C:\EEG DATA\020\concatenated\020_concatenated_raw.fif
Closing C:\EEG DATA\020\concatenated\020_concatenated_raw.fif
[done]
Saved to C:\EEG DATA\020\concatenated\020_concatenated_raw.fif
Processing 033...
Extracting parameters from C:\EEG DATA\033\eeg\TMR.vhdr...
Setting channel info structure...
Reading 0 ... 11859549  =      0.000 ... 23719.098 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Overwriting existing file.
Writing C:\EEG DATA\033\concatenated\033_concatenated_raw.fif
Closing C:\EEG DATA\033\concatenated\033_concatenated_raw.fif
[done]
Saved to C:\EEG DATA\033\concatenated\033_concatenated_raw.fif
Processing 046...
Extracting parameters from C:\EEG 

In [10]:
participants = ["032", "019"]
base_dir = r"C:\EEG DATA"

for pid in participants:

    print(f"Processing {pid}...")

    # Load raw EEG
    filepath = os.path.join(base_dir, pid, "eeg", "TMR.vhdr")
    raw = load_eeg_data(filepath)

    # Run analysis pipeline
    if pid in label_data_onsets:
        onset_values = label_data_onsets[pid]
        groups = group_by_increment(onset_values)
        segments = extract_segments(raw, groups)
        combined_raw = concatenate_segments(segments)

        if combined_raw is not None:
            save_dir = os.path.join(base_dir, pid, "concatenated")
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f"{pid}_concatenated_raw.fif")
            combined_raw.save(save_path, overwrite=True)
            print(f"Saved to {save_path}")
        else:
            print(f"No valid segments to save for {pid}")
    else:
        print(f"No onset data for {pid}")

# example of save path (participant 033)
# C:\EEG DATA\033\concatenated\033_concatenated_raw.fif

Processing 032...
Extracting parameters from C:\EEG DATA\032\eeg\TMR.vhdr...
Setting channel info structure...
Reading 0 ... 14187139  =      0.000 ... 28374.278 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Writing C:\EEG DATA\032\concatenated\032_concatenated_raw.fif
Closing C:\EEG DATA\032\concatenated\032_concatenated_raw.fif
[done]
Saved to C:\EEG DATA\032\concatenated\032_concatenated_raw.fif
Processing 019...
Extracting parameters from C:\EEG DATA\019\eeg\TMR.vhdr...
Setting channel info structure...
Reading 0 ... 10932739  =      0.000 ... 21865.478 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Writing C:\EEG DATA\019\concatenated\019_concatenated_raw.fif
Closing C:\EEG DATA\019\concatenated\019_concatenated_raw.fif
[done]
Saved to C:\EEG DATA\019\concatenated\019_concatenated_raw.fif
